In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
# Downloading daily gold futures data for the last 3 months
data = yf.download("GC=F", period="3mo", interval="1d")
close_prices = data['Close']['GC=F']
close_prices.head()

/var/folders/q3/9v6_7yk17_3gzz626ngfv51h0000gn/T/ipykernel_4713/1864209210.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download("GC=F", period="3mo", interval="1d")
[*********************100%***********************]  1 of 1 completed


Date
2025-08-27    3404.600098
2025-08-28    3431.800049
2025-08-29    3473.699951
2025-09-02    3549.399902
2025-09-03    3593.199951
Name: GC=F, dtype: float64

In [3]:
returns = np.log(close_prices / close_prices.shift(1)).dropna()
mu = returns.mean()
sigma = returns.std()

pd.DataFrame({
    'Average daily return': [mu],
    'Daily volatility': [sigma]
})

,Average daily return,Daily volatility
0,0.003413,0.01541


In [4]:
S0 = close_prices.iloc[-1]  # last closing price
T = 5                        # simulate 5 days
steps_per_day = 24            # hourly steps
steps = T * steps_per_day
paths = 5000                  # number of simulation paths

dt = 1 / steps_per_day
sigma_hourly = sigma / np.sqrt(steps_per_day)
mu_hourly = mu / steps_per_day

simulations = np.zeros((steps, paths))
simulations[0] = S0

In [ ]:
# Geometric Borwnian Motion formula
for t in range(1, steps):
    Z = np.random.normal(size=paths)
    simulations[t] = simulations[t-1] * np.exp((mu_hourly - 0.5*sigma_hourly**2)*dt + sigma_hourly*np.sqrt(dt)*Z)

simulations.shape

(120, 5000)

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(simulations, color='lightblue', alpha=0.2)  # all paths faint
plt.plot(simulations.mean(axis=1), color='red', label='Average path')  # mean path
plt.title("Monte Carlo Simulation: GC=F (Gold Futures)")
plt.xlabel("Hourly steps")
plt.ylabel("Price")
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(8,5))
plt.hist(simulations[-1], bins=50, density=True, color='orange', alpha=0.7)
plt.title("Probability Distribution of Gold Price at Day 5")
plt.xlabel("Price")
plt.ylabel("Probability Density")
plt.show()

In [6]:
pd.Series(simulations[-1]).describe()

count    5000.000000
mean     4224.133311
std        29.445462
min      4110.561598
25%      4203.885882
50%      4224.144771
75%      4243.856147
max      4338.563126
dtype: float64

In [ ]:
from matplotlib.colors import LogNorm
# High price bins
bins = np.linspace(simulations.min(), simulations.max(), 200)

# Heatmap matrix (bins x time steps)
heatmap_data = np.zeros((len(bins) - 1, simulations.shape[0]))

# Density histograms for every time step
for t in range(simulations.shape[0]):
    counts, _ = np.histogram(simulations[t], bins=bins, density=True)
    heatmap_data[:, t] = counts

plt.figure(figsize=(16,6))

# LogNorm enhances contrast in low-density regions
sns.heatmap(
    heatmap_data,
    cmap='icefire_r',
    cbar=True,
    norm=LogNorm(),   # better gradient visibility
)

plt.xlabel("Future Hour Steps")
plt.ylabel("Price Bins")
plt.title("Monte Carlo Probability Heatmap (High Resolution)")
plt.tight_layout()
plt.show()

In [ ]:
# Short and Long EMA periods
short_window = 10  # e.g., 10 days
long_window = 30   # e.g., 30 days

# Computing EMAs on historical close prices
ema_short = close_prices.ewm(span=short_window, adjust=False).mean()
ema_long = close_prices.ewm(span=long_window, adjust=False).mean()

# Plotting EMAs with historical price
plt.figure(figsize=(12,6))
plt.plot(close_prices, label='Close Price', color='black')
plt.plot(ema_short, label=f'EMA {short_window}', color='blue')
plt.plot(ema_long, label=f'EMA {long_window}', color='red')
plt.title("Gold Futures Price with EMA Crossovers")
plt.xlabel("Date")
plt.ylabel("Price")
plt.legend()
plt.show()